
# BLU14 - Exercise Notebook


In [ ]:
import os
import base64
import pandas as pd
import numpy as np
import category_encoders
import json
import joblib
import pickle
import requests
import seaborn as sns
from uuid import uuid4
from copy import deepcopy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['figure.figsize']=(4.8,3.6)

After the police, you nailed another big client. A bank hired you to assess whether or not a person is potentially a good client. For that purpose, they want you to design a system that tries to predict if a given individual earns more than 50K a year. You're no expert in the financial field, but you decide to take on the challenge. They provide you with a dataset of existing clients they have and their earnings.

<img src="media/banks.png" width=350 />

They also provide you with the following data description:

    1) age - client's age
    2) workclass - type of work performed by the client (eg. `Private`)
    3) fnlwgt - final weight assigned by the Census Bureau: if two samples have the same (or similar) fnlwgt they have similar characteristics, demographically speaking
    4) education - level of education of client (eg. `Bachelors`)
    5) education-num - numerically encoded level of education
    6) marital-status - client's marital status (eg `Widowed`)
    7) occupation - type of job held by the client (eg. `Craft-repair`)
    8) relationship - family position of the client
    9) race - client's race
    10) sex - "male"/"female"
    11) capital-gain - total capital gain in the previous year
    12) capital-loss - total capital loss in previous year
    13) hours-per-week - number of hours the the client works per week
    14) native-country - client's original nationality (eg. `Portugal`)

Load the dataset below and check out its format:

In [ ]:
def load_data():
    df = pd.read_csv(os.path.join("data", "bank.csv"))
    return df

df = load_data()
df.head()

Let's split our data into train and test:

In [ ]:
df = load_data()
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

df_test.salary.value_counts().plot(kind="bar");
plt.xlabel('Target value');
plt.ylabel('Target value counts');

## Exercise 1 - The model
In this exercise, we prepare a model and check if it meets the client's requirements.

### Exercise 1.1 - Train and serialize a baseline model

Build a baseline model for this problem (don't worry about performance for now) and serialize it. Use the following features:

    1) age 
    2) workclass - type of work performed by the client (eg. `Private`)
    4) education - level of education of client (eg. `Bachelors`)
    6) marital-status - client's marital status (eg `Widowed`)
    9) race - client's race
    10) sex - "male"/"female"
    11) capital-gain - total capital gain in the previous year
    12) capital-loss - total capital loss in the previous year
    13) hours-per-week - number of hours the client works per week

Make sure to change the target so that it has a binary value - True or False - instead of the original values. In particular:

* False: client has as a salary of smaller or equal to 50K
* True: client has as a salary higher than 50K

Use `df_train` to train your model. The model and any preprocessing should be part of a pipeline. The pipeline should be able to receive a dataframe with the columns we've requested you to use in the form `pipeline.predict(test_df)`. If you use models or functions that have a random component, ensure that you pass a random state so that there are no surprises when you submit.

Serialize the model into these variables: `columns.json`, `dtypes.pickle` and `pipeline.pickle`.

In [ ]:
# This is a temporary directory where your serialized files will be saved. Make sure you use this as 
# the target folder when you serialize your files
TMP_DIR = '/tmp'

In [ ]:
# pipeline = ...

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with open(os.path.join(TMP_DIR, 'columns.json')) as fh:
    columns = json.load(fh)
    assert columns == ["age", "workclass", "education", "marital-status", "race", "sex", "capital-gain", "capital-loss", "hours-per-week"]

with open(os.path.join(TMP_DIR, 'dtypes.pickle'), 'rb') as fh:
    dtypes = pickle.load(fh)
    assert dtypes.apply(lambda x: str(x)).isin(["int64", "int32", "object"]).all()

with open(os.path.join(TMP_DIR, 'pipeline.pickle'), 'rb') as fh:
    pipeline = joblib.load(fh)
    assert isinstance(pipeline, Pipeline)
    assert pipeline.predict(pd.DataFrame([{
        "age": 23, 
        "workclass": "Private", 
        "education": "Bachelors", 
        "marital-status": "Widowed", 
        "race": "White",
        "sex": "male",
        "capital-gain": 0, 
        "capital-loss": 0, 
        "hours-per-week": 40}
    ], columns=columns).astype(dtypes)) in [0, 1]

### Exercise 1.2 - Client requirements

Now, the client asks you one more thing. They want to make sure your model is as good at retrieving male cases of high salary as it is retrieving female cases. 

For example, if we have a pool of clients where 100 male clients earn more than 50K and we retrieve 80 out of those, and where 100 female clients also earn more than 50K but we only return 20 from those, then you're discriminating and that's not ok. A similar proportion, such as 75 women out of the 100 that earn higher, is expected.

Build a small function to calculate the retrieval rates of male and female clients. In particular, verify that the difference is not higher than 10%.

In [ ]:
def verify_retrieve_rates(X_test, y_true, y_test):
    """
    Calculate the retrieval rates for different `sex` instances
    and verify that they don't differ by more than 10 percentage points.
    
    Parameters:
        X_test (pd.DataFrame): features for the test cases
        y_true (pd.Series): true labels for the test cases [0, 1]
        y_test (pd.Series): predictions for the test cases [0, 1]

    Returns: tuple of (success, rate_difference)
        success (bool): True if the condition is satisfied, otherwise False 
        rate_difference (float): difference between each class retrieval rates (as an absolute value) 
        
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
model_1 = pd.read_csv(os.path.join('data', 'data_model_1.csv'))
X_test = model_1.drop(columns=['target', 'prediction'])
y_test = model_1.target
y_pred = model_1.prediction

success, rate_diff = verify_retrieve_rates(X_test, y_test, y_pred) 
assert isinstance(success,bool), 'The success variable should be a boolean.'
assert not success, 'Not correct, did you use the correct metric?'
np.testing.assert_almost_equal(rate_diff, 0.55, decimal=2, 
                               err_msg='Retrieval rate difference for model 1 is not correct.')

model_2 = pd.read_csv(os.path.join('data', 'data_model_2.csv'))
X_test = model_2.drop(columns=['target', 'prediction'])
y_test = model_2.target
y_pred = model_2.prediction

success, rate_diff = verify_retrieve_rates(X_test, y_test, y_pred) 
assert isinstance(success,bool), 'The success variable should be a boolean.'
assert success, 'Not correct, did you use the correct metric?'
np.testing.assert_almost_equal(rate_diff, 0.05, decimal=2, 
                               err_msg='Retrieval rate difference for model 2 is not correct.')

<img src="media/client-specs.png" width=400 />

## Exercise 2 - Prepare the model to be served
Here we'll deserialize the model from exercise 1 and prepare a prediction function for a server.

### Exercise 2.1 - Unpickle the model
Write a function that will unpickle all the pickled components - the column names, their dtypes, and the trained pipeline.

In [ ]:
def unpickle_model(path_to_dir, file_columns, file_dtypes, file_pipeline):
    """
    Unpickles model components.

    Parameters:
        path_to_dir: directory where the pickled files are stored
        file_columns: file name where the pickled columns are stored
        file_dtypes: file name where the pickled dtypes are stored
        file_pipeline: file name where the pickled model is stored
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return columns, dtypes, pipeline

In [ ]:
columns, dtypes, pipeline = unpickle_model(TMP_DIR, 'columns.json', 'dtypes.pickle', 'pipeline.pickle')
assert columns == ["age", "workclass", "education", "marital-status", "race", "sex", "capital-gain", "capital-loss", "hours-per-week"]
assert dtypes.apply(lambda x: str(x)).isin(["int64", "int32", "object"]).all()
assert isinstance(pipeline, Pipeline)
assert pipeline.predict(pd.DataFrame([{
        "age": 23, 
        "workclass": "Private", 
        "education": "Bachelors", 
        "marital-status": "Widowed", 
        "race": "White",
        "sex": "male",
        "capital-gain": 0, 
        "capital-loss": 0, 
        "hours-per-week": 40}
    ], columns=columns).astype(dtypes)) in [0, 1]

### Exercise 2.2 - Prediction from a request
Now use the unpickled model and build a predict function around it that will parse a request and return the respective prediction. The prediction should be True or False. Do not worry about potential bad inputs at this point, we'll get to it later on.

In [ ]:
def predict(columns, dtypes, pipeline, request):
    """
    Produce prediction for request.
    
    Parameters:
        columns: unpickled column name for the data in the request
        dtypes: unpickled column dtypes
        pipeline: unpickled model pipeline
        request: dictionary with the format described below
        {
            "observation_id": <id-as-a-string>,
            "data": {
                "age": <value>,
                "sex": <value>,
                "race": <value>,
                "workclass": <value>,
                "education": <value>,
                "marital-status": <value>,
                "capital-gain": <value>,
                "capital-loss": <value>,
                "hours-per-week": <value>,
            }
        }
    
    Returns:
        response: A dictionary echoing the request and its data with the addition of the prediction and probability 
            {
                "observation_id": <id-of-request>,
                "age": <value-of-request>,
                "sex": <value-of-request>,
                "race": <value-of-request>,
                "workclass": <value-of-request>,
                "education": <value-of-request>,
                "marital-status": <value-of-request>,
                "capital-gain": <value-of-request>,
                "capital-loss": <value-of-request>,
                "hours-per-week": <value-of-request>,
                "prediction": <True|False>,
                "probability": <probability generated by model>
            }
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return response

In [ ]:
request = {
    "observation_id": "1",
    "data": 
        {
            "age": 30, 
            "workclass": "Private", 
            "sex": "Female", 
            "race": "Amer-Indian-Eskimo", 
            "education": "Masters", 
            "marital-status": "Never-married", 
            "capital-gain": 0, 
            "capital-loss": 0, 
            "hours-per-week": 45, 
        }
}

response = predict(columns, dtypes, pipeline, request)
assert sorted(response.keys()) == \
    sorted(["observation_id", "age", "sex", "race", "education", "marital-status", "workclass",
            "capital-gain", "capital-loss", "hours-per-week", "prediction", "probability"])

assert response["observation_id"] == "1"
assert response["age"] == 30
assert response["hours-per-week"] == 45
assert response["prediction"] in [True, False]
probability_1 = response["probability"] 

request = {
    "observation_id": "2",
    "data": 
        {
            "age": 44, 
            "workclass": "Private", 
            "sex": "Male", 
            "race": "White", 
            "education": "Some-college", 
            "marital-status": "Married-civ-spouse", 
            "capital-gain": 0, 
            "capital-loss": 0, 
            "hours-per-week": 40, 
        }
}

response = predict(columns, dtypes, pipeline, request)
assert sorted(response.keys()) == \
    sorted(["observation_id", "age", "sex", "race", "education", "marital-status", "workclass",
            "capital-gain", "capital-loss", "hours-per-week", "prediction", "probability"])

assert response["observation_id"] == "2"
assert response["education"] == "Some-college"
assert response["hours-per-week"] == 40
assert response["prediction"] in [True, False]
probability_2 = response["probability"] 
assert probability_1 != probability_2

Hurray! It passed the tests. 

## Exercise 3 - Protecting the server

Let's be a bit more thorough with our server. To avoid issues and ensure that we have full control around what is going on, we need to reason about which values we expect to receive and in what format.

<img src="media/darth-vader-validation.jpg" width=400 />

### Exercise 3.1 - Categorical values

First, we'll think about categorical values. As the name indicates, these are the values that are restricted to a set of potential choices. So logically, what we want to verify when one of these arrives at our server is that they belong to the correct categories. 

Create a function that returns a list of all possible values of a given categorical column in a dataframe.

In [ ]:
def get_valid_categories(df, column):
    """
    Obtains the list of available categories for column.
    
    Inputs:
        df (pandas.DataFrame): dataframe from which to extract column values
        column (str): categorical column for which to extract values
    
    Returns:
        categories: A list of potential values for the column
    """

    # YOUR CODE HERE
    raise NotImplementedError()
    return categories

In [ ]:
df = load_data()

# Test dataframe categorical values
assert get_valid_categories(df, "sex") == ["Male", "Female"]
assert get_valid_categories(df, "race") ==  ["White", "Black", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other"]
assert len(get_valid_categories(df, "workclass")) == 9
assert len(get_valid_categories(df, "education")) == 16
assert len(get_valid_categories(df, "marital-status")) == 7

# Test dataframe numerical values - notice the amount of potentially different values
assert len(get_valid_categories(df, "age")) == 71
assert len(get_valid_categories(df, "capital-loss")) == 83
assert len(get_valid_categories(df, "hours-per-week")) == 90
assert len(get_valid_categories(df, "capital-gain")) == 116

### Exercise 3.2 - Numerical values

Now we'll look into numerical values. We cannot use the same approach as for the categorical features because we don't know all the possible values. What we can do is check if the values fall into a reasonable range.

#### Exercise 3.2.1 - Age range

Which age range is the most adequate to use in this particular model?

- A) -100 to 100
- B) 0 to 100
- C) 20 to 1000

Enter your answer below as a string, for example:

```
answer_q32i = "A"
```

In [ ]:
# answer_ex_321 = 'A' or 'B' or 'C' 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert base64.b64encode(answer_ex_321.encode()) == b'Qg=='

#### Exercise 3.2.2 - Capital gain

However, not all features are that obvious and have ranges we can reason about. There are some strategies to go around it. Take for example the capital gain, what do you think is most appropriate?

- A) Taking the minimum and maximum of the observed values and using them as a range
- B) Setting fixed values - eg. 0 to 10000
- C) Leaving the range of allowed values completely free of any specification

Enter your answer below as a string, for example:

```
answer_ex_322 = "A"
```

In [ ]:
# answer_ex_322 = 'A' or 'B' or 'C' 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert base64.b64encode(answer_ex_322.encode()) == b'QQ=='

### Exercise 3.3 - Putting everything together

Now put everything together. Create a predict function similar to the one from exercise 2.2  and protect it against unexpected inputs. 

If everything is well with your request return an answer like this:

```json
{
    "observation_id": "id1234",
    "prediction": True,
    "probability": 0.4
}
```

However, if there is a problem with the initial data, such as missing fields or invalid values, we want to return a different response:

```json
{
    "observation_id": "id1234",
    "error": "Some error occured",
}
```

If the `observation_id` is not present, set it to None.

Check out the tests to see what we expect from the error cases and error messages. Even though we mentioned better strategies above for values such as capital-gain and capital-loss, it's enough here to protect against what the tests show.

In [ ]:
def attempt_predict(request):
    """
    Produce prediction for request.
    
    Inputs:
        request: dictionary with the format described below
        {
            "observation_id": <id-as-a-string>,
            "data": {
                "age": <value>,
                "sex": <value>,
                "race": <value>,
                "workclass": <value>,
                "education": <value>,
                "marital-status": <value>,
                "capital-gain": <value>,
                "capital-loss": <value>,
                "hours-per-week": <value>,
            }
        }
     
    Returns: A dictionary with predictions or an error, the two potential values:
                if the request is OK and was properly parsed and predicted:
                {
                    "observation_id": <id-of-request>,
                    "prediction": <True|False>,
                    "probability": <probability generated by model>
                }
                otherwise:
                {
                    "observation_id": <id-of-request>,
                    "error": "some error message"
                }
    """

    # YOUR CODE HERE
    raise NotImplementedError()
    return response

In [ ]:
# Initialization code
with open(os.path.join(TMP_DIR, 'columns.json')) as fh:
    columns = json.load(fh)

with open(os.path.join(TMP_DIR, 'dtypes.pickle'), 'rb') as fh:
    dtypes = pickle.load(fh)

with open(os.path.join(TMP_DIR, 'pipeline.pickle'), 'rb') as fh:
    pipeline = joblib.load(fh)

################################################
# Test with good payload
################################################
print('Test with good payload')
base_request = {
    "observation_id": "1",
    "data": 
        {
            "age": 30, 
            "workclass": "Private", 
            "sex": "Female", 
            "race": "Amer-Indian-Eskimo", 
            "education": "Masters", 
            "marital-status": "Never-married", 
            "capital-gain": 0, 
            "capital-loss": 0, 
            "hours-per-week": 45, 
        }
}

response = attempt_predict(base_request)
assert 'prediction' in response, response
assert 'probability' in response, response
assert 'observation_id' in response, response

assert response["observation_id"] == "1", response["observation_id"]
assert response["prediction"] in [True, False], response["prediction"] 
assert response["probability"] <= 1.0, response["probability"] 
assert response["probability"] >= 0.0, response["probability"] 


################################################
# Test missing `observation_id` produces an error
################################################
print('Test missing `observation_id` produces an error')
bad_request_1 = deepcopy(base_request)
bad_request_1['random_field'] = bad_request_1.pop('observation_id')

response = attempt_predict(bad_request_1)
assert 'error' in response, response
assert 'observation_id' in response['error'] 


################################################
# Test missing `data` produces an error
################################################
print('Test missing `data` produces an error')
bad_request_2 = deepcopy(base_request)
bad_request_2['data_field_name'] = bad_request_2.pop('data')

response = attempt_predict(bad_request_2)
assert 'error' in response, response
assert 'data' in response['error'] 


################################################
# Test missing columns produce an error
################################################
print('Test missing columns produce an error')
bad_request_3 = deepcopy(base_request)
bad_request_3['data'].pop('age')

response = attempt_predict(bad_request_3)
assert 'error' in response, response
assert 'age' in response['error'], response['error']


################################################
# Test extra columns produce an error
################################################
print('Test extra columns produce an error')
bad_request_4 = deepcopy(base_request)
bad_request_4['data']['relationship'] = "Wife"

response = attempt_predict(bad_request_4)
assert 'error' in response, response
assert 'relationship' in response['error'], response['error']

Run a couple more tests to make sure your server is bulletproof:

In [ ]:
####################################################
# Test invalid values for categorical features - sex
####################################################
print('Test invalid values for categorical features - sex')
bad_request_5 = deepcopy(base_request)
bad_request_5['data']['sex'] = "Engineeer"

response = attempt_predict(bad_request_5)
assert 'error' in response, response
assert 'sex' in response['error'], response['error']
assert 'Engineeer' in response['error'], response['error']


###########################################################################
# Test invalid values for categorical features - race
###########################################################################
print('Test invalid values for categorical features - race')
bad_request_6 = deepcopy(base_request)
bad_request_6['data']['race'] = 'Male'

response = attempt_predict(bad_request_6)
assert 'error' in response, response
assert 'race' in response['error'], response['error']
assert 'Male' in response['error'], response['error']


####################################################
# Test invalid values for numerical features - age
####################################################
print('Test invalid values for numerical features - age')
bad_request_7 = deepcopy(base_request)
bad_request_7['data']['age'] = -12

response = attempt_predict(bad_request_7)
assert 'error' in response, response
assert 'age' in response['error'], response['error']
assert '-12' in response['error'], response['error']

bad_request_8 = deepcopy(base_request)
bad_request_8['data']['age'] = 1200

response = attempt_predict(bad_request_8)
assert 'error' in response, response
assert 'age' in response['error'], response['error']
assert '1200' in response['error'], response['error']


####################################################
# Test invalid values for numerical features - capital gain and loss
####################################################
print('Test invalid values for numerical features - capital gain and loss')
bad_request_9 = deepcopy(base_request)
bad_request_9['data']['capital-gain'] = -10

response = attempt_predict(bad_request_9)
assert 'error' in response, response
assert 'capital-gain' in response['error'], response['error']
assert '-10' in response['error'], response['error']

bad_request_10 = deepcopy(base_request)
bad_request_10['data']['capital-loss'] = -500

response = attempt_predict(bad_request_10)
assert 'error' in response, response
assert 'capital-loss' in response['error'], response['error']
assert '-500' in response['error'], response['error']


####################################################
# Test invalid values for numerical features - hours per week
####################################################
print('Test invalid values for numerical features - hours per week')
bad_request_11 = deepcopy(base_request)
bad_request_11['data']['hours-per-week'] = -10

response = attempt_predict(bad_request_11)
assert 'error' in response, response
assert 'hours-per-week' in response['error'], response['error']
assert '-10' in response['error'], response['error']

bad_request_12 = deepcopy(base_request)
bad_request_12['data']['hours-per-week'] = 400

response = attempt_predict(bad_request_12)
assert 'error' in response, response
assert 'hours-per-week' in response['error'], response['error']
assert '400' in response['error'], response['error']

Ufff. That was tough. But now your app is a bit safer to deploy! At least from all the cases we could think of.

<img src="media/code-passes-tests.png" width=500 />

## Exercise 4 - Put everything together

Finally, build a server with your model and a predict endpoint protected from all the cases as in exercise 3. Deploy it and set 
the name of your app below:

In [ ]:
# Assign the variable APP_NAME to the name of your railway app
# APP_NAME = ...

# YOUR CODE HERE
raise NotImplementedError()

Test that your server is bulletproof. You can do the first testing locally, just don't forget to change the url for the remote one before submitting the notebook.

In [ ]:
# Test locally
# url = f"http://localhost:5000/predict"


# Testing the predict/update endpoint
url = "https://{}/predict".format(APP_NAME)


################################################
# Test with good payload
################################################
print('Test with good payload')
payload = {
    "observation_id": str(uuid4()),
    "data": 
        {
            "age": 30, 
            "workclass": "Private", 
            "sex": "Female", 
            "race": "Amer-Indian-Eskimo", 
            "education": "Masters", 
            "marital-status": "Never-married", 
            "capital-gain": 0, 
            "capital-loss": 0, 
            "hours-per-week": 45, 
        }
}


r = requests.post(url, json=payload)
assert isinstance(r, requests.Response)
assert r.ok

response = r.json()

assert 'prediction' in response, response
assert 'probability' in response, response

assert response["prediction"] in [True, False]

assert isinstance(response["probability"], float)
assert 0 <= response["probability"] <= 1


################################################
# Test missing `observation_id` produces an error
################################################
print('Test missing `observation_id` produces an error')
bad_payload_1 = deepcopy(payload)
bad_payload_1['random_field'] = bad_payload_1.pop('observation_id')

r = requests.post(url, json=bad_payload_1)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'observation_id' in response['error'], response['error']


################################################
# Test missing `data` produces an error
################################################
print('Test missing `data` produces an error')
bad_payload_2 = deepcopy(payload)
bad_payload_2['observation_id'] = str(uuid4())
bad_payload_2['random_field'] = bad_payload_2.pop('data')

r = requests.post(url, json=bad_payload_2)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'data' in response['error'], response['error']


################################################
# Test missing columns produce an error
################################################
print('Test missing columns produce an error')
bad_payload_3 = deepcopy(payload)
bad_payload_3['observation_id'] = str(uuid4())
bad_payload_3['data'].pop('age')

r = requests.post(url, json=bad_payload_3)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'age' in response['error'], response['error']


################################################
# Test extra columns produce an error
################################################
print('Test extra columns produce an error')
bad_payload_4 = deepcopy(payload)
bad_payload_4['observation_id'] = str(uuid4())
bad_payload_4['data']['relationship'] = "Wife"

r = requests.post(url, json=bad_payload_4)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'relationship' in response['error'], response['error']


###########################################################################
# Test invalid values for categorical features - race
###########################################################################
print('Test invalid values for categorical features - race')

bad_payload_5 = deepcopy(payload)
bad_payload_5['observation_id'] = str(uuid4())
bad_payload_5['data']['race'] = 'Engineer'

r = requests.post(url, json=bad_payload_5)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'race' in response['error'], response['error']
assert 'Engineer' in response['error'], response['error']


####################################################
# Test invalid values for numerical features - age
####################################################
print('Test invalid values for numerical features - age')
bad_payload_6 = deepcopy(payload)
bad_payload_6['observation_id'] = str(uuid4())
bad_payload_6['data']['age'] = -12

r = requests.post(url, json=bad_payload_6)
assert isinstance(r, requests.models.Response)
assert r.ok

response = r.json()

assert 'error' in response, response
assert 'age' in response['error'], response['error']
assert '-12' in response['error'], response['error']

And... you're done. You have successfully built a model, assessed if it passed the client requirements, built an app and protected it from crappy input. 

It's time for a well-deserved rest, so go ahead and be a couch potato.

<img src="media/lays.png" width=400 />